In [1]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader
import numpy as np

import io
import imageio
from ipywidgets import widgets, HBox

# Use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# Original ConvLSTM cell as proposed by Shi et al.
class ConvLSTMCell(nn.Module):

    def __init__(self, in_channels, out_channels, 
    kernel_size, padding, activation, frame_size):

        super(ConvLSTMCell, self).__init__()  

        if activation == "tanh":
            self.activation = torch.tanh 
        elif activation == "relu":
            self.activation = torch.relu
        
        # Idea adapted from https://github.com/ndrplz/ConvLSTM_pytorch
        self.conv = nn.Conv2d(
            in_channels=in_channels + out_channels, 
            out_channels=4 * out_channels, 
            kernel_size=kernel_size, 
            padding=padding)           

        # Initialize weights for Hadamard Products
        self.W_ci = nn.Parameter(torch.Tensor(out_channels, *frame_size))
        self.W_co = nn.Parameter(torch.Tensor(out_channels, *frame_size))
        self.W_cf = nn.Parameter(torch.Tensor(out_channels, *frame_size))

    def forward(self, X, H_prev, C_prev):

        # Idea adapted from https://github.com/ndrplz/ConvLSTM_pytorch
        conv_output = self.conv(torch.cat([X, H_prev], dim=1))

        # Idea adapted from https://github.com/ndrplz/ConvLSTM_pytorch
        i_conv, f_conv, C_conv, o_conv = torch.chunk(conv_output, chunks=4, dim=1)

        input_gate = torch.sigmoid(i_conv + self.W_ci * C_prev )
        forget_gate = torch.sigmoid(f_conv + self.W_cf * C_prev )

        # Current Cell output
        C = forget_gate*C_prev + input_gate * self.activation(C_conv)

        output_gate = torch.sigmoid(o_conv + self.W_co * C )

        # Current Hidden State
        H = output_gate * self.activation(C)

        return H, C

In [3]:
class ConvLSTM(nn.Module):

    def __init__(self, in_channels, out_channels, 
    kernel_size, padding, activation, frame_size):

        super(ConvLSTM, self).__init__()

        self.out_channels = out_channels

        # We will unroll this over time steps
        self.convLSTMcell = ConvLSTMCell(in_channels, out_channels, 
        kernel_size, padding, activation, frame_size)

    def forward(self, X):

        # X is a frame sequence (batch_size, num_channels, seq_len, height, width)

        # Get the dimensions
        batch_size, _, seq_len, height, width = X.size()

        # Initialize output
        output = torch.zeros(batch_size, self.out_channels, seq_len, 
        height, width, device=device)
        
        # Initialize Hidden State
        H = torch.zeros(batch_size, self.out_channels, 
        height, width, device=device)

        # Initialize Cell Input
        C = torch.zeros(batch_size,self.out_channels, 
        height, width, device=device)

        # Unroll over time steps
        for time_step in range(seq_len):

            H, C = self.convLSTMcell(X[:,:,time_step], H, C)

            output[:,:,time_step] = H

        return output

In [4]:
class Seq2Seq(nn.Module):

    def __init__(self, num_channels, num_kernels, kernel_size, padding, 
    activation, frame_size, num_layers):

        super(Seq2Seq, self).__init__()

        self.sequential = nn.Sequential()

        # Add First layer (Different in_channels than the rest)
        self.sequential.add_module(
            "convlstm1", ConvLSTM(
                in_channels=num_channels, out_channels=num_kernels,
                kernel_size=kernel_size, padding=padding, 
                activation=activation, frame_size=frame_size)
        )

        self.sequential.add_module(
            "batchnorm1", nn.BatchNorm3d(num_features=num_kernels)
        ) 

        # Add rest of the layers
        for l in range(2, num_layers+1):

            self.sequential.add_module(
                f"convlstm{l}", ConvLSTM(
                    in_channels=num_kernels, out_channels=num_kernels,
                    kernel_size=kernel_size, padding=padding, 
                    activation=activation, frame_size=frame_size)
                )
                
            self.sequential.add_module(
                f"batchnorm{l}", nn.BatchNorm3d(num_features=num_kernels)
                ) 

        # Add Convolutional Layer to predict output frame
        self.conv = nn.Conv2d(
            in_channels=num_kernels, out_channels=num_channels,
            kernel_size=kernel_size, padding=padding)

    def forward(self, X):

        # Forward propagation through all the layers
        output = self.sequential(X)

        # Return only the last output frame
        output = self.conv(output[:,:,-1])
        
        return nn.Sigmoid()(output)

In [5]:
# Load Data as Numpy Array
# MovingMNIST = np.load('/kaggle/input/mnist-moving/mnist_test_seq.npy')
MovingMNIST = np.load('/kaggle/input/our-dummy-dataset/merged.npy')
print(MovingMNIST.shape)

# MovingMNIST = MovingMNIST.transpose(1, 0, 2, 3)
# print(MovingMNIST.shape)

# Shuffle Data
np.random.shuffle(MovingMNIST)

# 70:15:15 split
n = len(MovingMNIST)
train_data, val_data, test_data = MovingMNIST[:int(0.7*n)], MovingMNIST[int(0.7*n):int(0.85*n)], MovingMNIST[int(0.85*n):]

f = 15 # total frames for training
def collate(batch):

    # Add channel dim, scale pixels between 0 and 1, send to GPU
    batch = torch.tensor(batch)
#     batch = batch / 255.0                        
    batch = batch.to(device)   
    
    # Randomly pick f frames as input, (f+1)th frame is target
    rand = np.random.randint(0, batch.shape[2] - f)  # Ensure there are enough frames
    return batch[:, :, rand:rand+f], batch[:, :, rand+f]

# Training Data Loader
train_loader = DataLoader(train_data, shuffle=True, 
                        batch_size=16, collate_fn=collate)

# Validation Data Loader
val_loader = DataLoader(val_data, shuffle=True, 
                        batch_size=16, collate_fn=collate)

(587, 2, 20, 64, 64)


In [6]:
# # Get a batch
# input, _ = next(iter(val_loader))
# print(input.shape)

# # Reverse process before displaying
# input = input.cpu().numpy() * 255.0     

# for video in input.squeeze(1)[:3]:          # Loop over videos
#     with io.BytesIO() as gif:
#         imageio.mimsave(gif,video.astype(np.uint8),"GIF",fps=5)
#         display(HBox([widgets.Image(value=gif.getvalue())]))

In [7]:
# Get a batch
input, _ = next(iter(val_loader))
print(input.shape)
print(_.shape)

# Reverse process before displaying
input = input.cpu().numpy() * 255.0

input = np.transpose(input, (0, 2, 3, 4, 1))

for video in input[:2]:  # Loop over first 3 videos
    frames = []
    for frame in video:
        # Take only the first channel
        frame = frame[:,:,0]
        frames.append(frame.astype(np.uint8))
    
    with io.BytesIO() as gif:
        imageio.mimsave(gif, frames, format="GIF", fps=1)
        display(HBox([widgets.Image(value=gif.getvalue())]))

/tmp/ipykernel_34/2379469153.py:20: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:261.)
  batch = torch.tensor(batch)


torch.Size([16, 2, 15, 64, 64])
torch.Size([16, 2, 64, 64])


In [18]:
# The input video frames are grayscale, thus single channel
model = Seq2Seq(num_channels=2, num_kernels=64, 
kernel_size=(3, 3), padding=(1, 1), activation="relu", 
frame_size=(64, 64), num_layers=6).to(device)

optim = Adam(model.parameters(), lr=1e-4)

# Binary Cross Entropy, target pixel values either 0 or 1
criterion = nn.BCEWithLogitsLoss(reduction='sum')

In [19]:
# clears gpu memory
import gc
gc.collect()

with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:
from tqdm import tqdm
import matplotlib.pyplot as plt

min_val_loss = 100000
num_epochs = 2

train_losses = []
val_losses = []

for epoch in tqdm(range(1, num_epochs+1), desc="Epochs"):
    train_loss = 0
    model.train()
    for batch_num, (input, target) in enumerate(train_loader, 1):
        output = model(input)
        loss = criterion(output.flatten(), target.flatten())
        loss.backward()
        optim.step()
        optim.zero_grad()
        train_loss += loss.item()
    
    train_loss /= len(train_loader.dataset)
    train_losses.append(train_loss)
    
    val_loss = 0
    model.eval()
    with torch.no_grad():
        for input, target in val_loader:
            output = model(input)
            loss = criterion(output.flatten(), target.flatten())
            val_loss += loss.item()
    
    val_loss /= len(val_loader.dataset)
    val_losses.append(val_loss)
    
    # save the model
    if val_loss < min_val_loss:
        min_val_loss = val_loss
        torch.save(model.state_dict(), "/kaggle/working/model.pth")
    
    print(f"Epoch:{epoch} Training Loss:{train_loss:.2f} Validation Loss:{val_loss:.2f}\n")
    
    # Plot loss graph
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, epoch+1), train_losses, label='Training Loss')
    plt.plot(range(1, epoch+1), val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.show()

Epochs:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def collate_test(batch):

    # Last 5 frames are target
    target = np.array(batch)[:,:,-5:]       
    
    # Add channel dim, scale pixels between 0 and 1, send to GPU
#     batch = torch.tensor(batch).unsqueeze(1)          
    batch = torch.tensor(batch) 
#     batch = batch / 255.0                             
    batch = batch.to(device)                          
    return batch, target

# Test Data Loader
test_loader = DataLoader(test_data,shuffle=True, 
                         batch_size=3, collate_fn=collate_test)

# Get a batch
batch, target = next(iter(test_loader))
print(batch.shape)
print(target.shape)

# Initialize output sequence
output = np.zeros(target.shape, dtype=np.uint8)

# Loop over timesteps
with torch.no_grad():
    for timestep in range(target.shape[2]):
      input = batch[:,:,timestep:timestep+15]   
      output[:,:,timestep] = (model(input).detach().cpu() * 255.0).numpy()


In [ ]:
target = np.transpose(target, (0, 2, 3, 4, 1))
output = np.transpose(output, (0, 2, 3, 4, 1))

for tgt, out in zip(target, output):       # Loop over samples
    # Write target video as gif
    
    tgt = tgt * 255.0
    
    frames = []
    for frame in tgt:
        # Take only the first channel
        frame = frame[:,:,0]
        frames.append(frame.astype(np.uint8))
        
    with io.BytesIO() as gif:
        imageio.mimsave(gif, frames, "GIF", fps = 1)    
        target_gif = gif.getvalue()

    # Write output video as gif
    frames = []
    for frame in out:
        # Take only the first channel
        frame = frame[:,:,0]
        frames.append(frame.astype(np.uint8))
        
    with io.BytesIO() as gif:
        imageio.mimsave(gif, frames, "GIF", fps = 1)    
        output_gif = gif.getvalue()

    display(HBox([widgets.Image(value=target_gif), 
                  widgets.Image(value=output_gif)]))